# Final set up for data collection 
### [RN] 2/16/21 12:10

Data collection has been running for two days. I have data up to Hour_63. Resulting data and all code can be found in [this](https://github.com/rnnisi/CosmicRayDetector_LongTerm/blob/main/InitialRun.tar.gz) zip file. 

I want to automate data updating and change the filing system to go by date and hour. I also need to move the detector to a better location, as the scintillating blocks were jostled and this data is not reliable. 

First I will edit the filing system. 


In [ ]:
### New function to generate file names based on date instead of sequentially numbered

def NameRun():
    d = datetime.datetime.now()
    M = d.strftime("%m")
    D = d.strftime("%d")
    H = str(int(float(d.hour) + 1))
    ID = M + "-" + D + "-" + H
    OutDir = ID + "-DATA"
    return OutDir, ID

Now I want to automate a filing system and upload so that my  pi does not get over run. I think the best way to do this is to create a whole new directory within one parent each day, and then delete older days for storage purposes. I will make a shell script to do this. 

i will work from a new directory called center, which has the controls. 

Now i can automatically run a shell script that makes a new directory each day at 11:55 and changes the path so that data writes to the directory for each day. 



In [ ]:
### Automates filing system by day

RAWD=$(date +%D)
TITLE=$(echo $RAWD | sed -e 's/\//\-/g')
mkdir $TITLE

NewLine='ParentDir = "'$TITLE'" #UPDATE_ParentDir'

sed -i "s/.*UPDATE_ParentDir.*/$NewLine/g" /home/pi/Desktop/CosmicRayDetection/center/1hr_CosmicRayCollect.py


To have this run, i am setting 1hr_CosmicRayCollect.py to run at every 59th minute on the hour. 1hr_CosmicRayCollect.py will get configure output files, then wait to start collecting data until it has been 60 seconds since initialization, which should be the top of the hour. It will leave an INSERT flag at configuration. Incident coscmic rays will be detected and recorded by time in count in the "Raw.txt" file in the new directory created for that hour. After exactly one hour, data collection will stop and the weather will be read out using the scraper and added to the data files. Total data sheet will also be updated. Crontab will keep these scripts running over time for continous data collection despite processing times thanks to the multiprocessing approach. 


There is not an easy and quick way to update data, so I will just do it manually to my git repo. 

I am goign to set up the detector somehwere out of the way where it can just read data for a long time protected from light 

### [RN] 2:48 

Had to go to office hours for a bit. I am now ready to set up the detector and leave it. I will have to manually upload and remove data. This is fine it is easy to ssh into the pi and check that it is working. 

The detectors are stacked vertically and wrapped in a sweatshirt. Pi and latcher board are mounted to avoid cross wires, movement, and so I can see lights. The whole thing is plugged in under the entry table in myhouse by the router. We will avoid moving it. I will note any disturbances or power outages. 


### [RN] 3:05

I am now setting up crontab. I am running a quick test to make sure the scripts are working appropriately first. 

I also am updating my update parent dir so that i can run a crontab to make a new directory for the day and update the pat 11:55 pm. Then the script running at 11:59 will output to the next day directory. 




In [ ]:
### my crontab 
rnnishi:/home/pi/Desktop/CosmicRayDetection/center$ crontab -l
PATH=/home/pi/.local/bin:/home/pi/physrpi/bin:/home/pi/physrpi/scripts:/home/pi/physrpi/mazin/bin:/home/pi/physrpi/mazin/scripts:/home/pi/physrpi/campagnari/bin:/home/pi/physrpi/campagnari/scripts:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games:.

SHELL=/bin/bash

LOGNAME=pi

PYTHONPATH=/home/pi/physrpi/python:/home/pi/physrpi/plib:/home/pi/physrpi/mazin/python:/home/pi/physrpi/campagnari/python

59 * * * * /home/pi/Desktop/CosmicRayDetection/center/1hr_CosmicRayCollect.py > /dev/null 2>&1
55 23 * * * /home/pi/Desktop/CosmicRayDetection/center/UpdateParentDir.sh > /dev/null 2>&1

In [ ]:
### 1hr_CosmicRayCollect.py
#!/usr/bin/env python3
  
import scipy.stats as stats
import numpy  as  np
import sys
import os
import subprocess
from threading import Thread
from multiprocessing import Process, Pipe
import _thread
import RPi.GPIO as GPIO
import time
import datetime
import sys

### Configure
st = time.perf_counter()
WaitToCollect = 60 # start at 59 minute
CollectTime = 60*60 # one hour
ParentDir = "02-16-21" #UPDATE_ParentDir
path = "/home/pi/Desktop/CosmicRayDetection/center/"
DataPath = path + ParentDir + "/"

### Functions
def CheckDir(f, ff):
    i = 2               # format for f should be : NAME_n.txt with n = 1, 2, 3.....
    while True:
        try:
            ls = subprocess.check_output(['ls ' + path + "/" + f + " >/dev/null 2>&1"], shell = True)
            f = eval(ff)
            i = i + 1
        except:
            break
    out = str(path + "/" + f)
    exp = str(i - 1)
    return out, exp

def NameRun():
    d = datetime.datetime.now()
    M = d.strftime("%m")
    D = d.strftime("%d")
    H = str(int(float(d.hour) + 1))
    ID = M+D+"_Hour" + H
    return ID

def TimeOut(st, duration, RawDat, child_conn):
    print(time.perf_counter() - st)
    time.sleep(duration - (time.perf_counter() - st))   # set timeout for operations defined in this class
    print("ending program", time.perf_counter() - st)
    done=True
    child_conn.close()
    RawDat.close() # close df
def CollectData(st, child_conn, RawDat):
    count = 0
    to = Thread(target = TimeOut, args=(st, CollectTime, RawDat, child_conn,)) # i want to collect data in precise time window
    to.setDaemon(True)
    to.start() # set timeout 
    while done==False:
        try:
            if GPIO.input(37)==1:
                count = count + 1
                currentTime = datetime.datetime.now()
                line = str(count) + ", " + str(time.perf_counter() - st) + ", " + str(currentTime) + "\n"
                RawDat.write(line)
                while GPIO.input(37)==1:
                    time.sleep(0.00001)
        except: # timeout kicked in during collection 
            print("Last attempt to send data: t = " + str(time.perf_counter() - st))
            to.join()
            os._exit(0)

# Setup Raw Data file
ID = NameRun()
OutDir = DataPath + "/" + ID
subprocess.Popen(["mkdir " + OutDir], shell=True)
subprocess.check_output(["ls " + OutDir], shell=True)
RawDat = OutDir + "/Raw.csv"
print(RawDat)

subprocess.check_output([path + "/StartDAQ.sh " + RawDat], shell=True) # get most recent weather data
print("Got weather data")

# get ready for data acq
WriteRaw = open(RawDat, 'a+')

GPIO.setmode(GPIO.BOARD)
GPIO.setup(37,GPIO.IN)

global done # flag 
done=False

# Setup DAQ as child process so it can be interrupted as soon as DAQ time is up; avoid duplicate data points
parent_conn, child_conn = Pipe() # multiprocessing to avoid timing issues

# wait until exact time to start. This is so i can start the initialization ahead of time and be ready to get precise time bins.
while time.perf_counter() - st <= WaitToCollect: # wait from initialization to start DAQ 
    pass
# start data collection as child process
DAQ_st = time.perf_counter()
GetData = Process(target = CollectData, args = (DAQ_st, child_conn, WriteRaw,)) # Data collection as child process

print(DAQ_st)
GetData.start()
# join when done 
GetData.join()
print(time.perf_counter() - DAQ_st)
print("proceed")

### Add weather to df header 

subprocess.check_output([path + "/AddWeather.sh " + RawDat + " > /dev/null"], shell = True)

### Process data


def ReadFile(infile):
    read = open(infile, 'r')
    lines = read.readlines()
    read.close()
    return lines

raw = ReadFile(RawDat)
print(raw)
event_rt = []
for i in range(7, len(raw)):
    temp = raw[i].split(', ')
    print(temp)
    event_rt.append(float(temp[1])/60)


EPM = [] # events per minute
minutes = 0
count = 0
print(event_rt)
for i in range(len(event_rt)):
    if event_rt[i] - minutes < 1:
        count = count + 1
    else:
        minutes = minutes + 1
        EPM.append(count)
        count = 1
EPM.append(count) # last unfill bin 

# make header for new df for processed data 
ProcDat = OutDir + "/EventsPerMin.csv"
subprocess.Popen(["head -4 " + RawDat + " > " + ProcDat], shell=True)
subprocess.check_output(["ls > /dev/null"], shell = True)

#write out events per minute
WriteEPM = open(ProcDat, 'a+')
WriteEPM.write("Minute since start, Events per minute\n")


for i in range(len(EPM)):
    print(EPM[i])
    WriteEPM.write(str(i+1) + ", " + str(EPM[i]) + "\n")
WriteEPM.close()

# Add to total file

EnvCond = raw[2].strip()

avg = stats.tmean(EPM)
std = stats.tstd(EPM)
var = stats.tvar(EPM)
rng = max(EPM) - min(EPM)

TotalDat = path + "/TotalOverTime.csv"
WriteTotal = open(TotalDat, 'a+')
DataSummary = str(avg) + ", " + str(std) + ", " + str(var) + ", " + str(rng)
WriteTotal.write("--- " +  str(ID) + " --- DATA: " + DataSummary + " --- CONDITIONS: " + EnvCond + "\n")
WriteTotal.close()
                     
    

In [ ]:
### AddWeather.sh

#!/bin/bash
  
# scrape weather data from first line in table of url = https://forecast.weather.gov/data/obhistory/metric/KSBA.html

URL="https://forecast.weather.gov/data/obhistory/metric/KSBA.html"
RAW=$(curl $URL )
CUT=$(echo $RAW | grep -oP '(?<=</table></form>).*?(?=</th></tr></table>)' | grep -oP '(?<=</th><th).*?(?=Time<br>)')
echo $CUT
echo 
TIME=$(echo $CUT | grep -oP '(?<=align="right">).*?(?=</td><td>)' | cut -d' ' -f1)
NOW=$(echo $TIME | cut -d' ' -f1)
echo "time"
echo $NOW

WC=$(echo $CUT | grep -oP '(?<=align="left">).*?(?=<td>)' | cut -d'>' -f1)
echo $WC
WN=$(echo $WC | cut -d'<' -f1)
echo "weather conditions"
echo $WN

TABLE=$(echo $RAW| grep -oP '(?<=/th>).*?(?=</th>)' )
echo "table"
echo $TABLE 

LINE=$(echo $TABLE | sed -e 's/\ //g' | grep -oP '(?<=<td>).*?(?=/td>)')
echo 
echo "line"
echo $LINE

DATE=$(echo $LINE | cut -d' ' -f1 | sed -e 's/<//g')
WIND=$(echo $LINE | cut -d' ' -f2 | sed -e 's/<//g')
VIS=$(echo $LINE | cut -d' ' -f3 | sed -e 's/<//g')
SKY_COND=$(echo $LINE | cut -d' ' -f4 | sed -e 's/<//g')
AIR_T=$(echo $LINE | cut -d' ' -f5 | sed -e 's/<//g')
DWPNT=$(echo $LINE | cut -d' ' -f6 | sed -e 's/<//g')
MAX6HR=$(echo $LINE | cut -d' ' -f7 | sed -e 's/<//g')
MIN6HR=$(echo $LINE | cut -d' ' -f8 | sed -e 's/<//g')
HUMID=$(echo $LINE | cut -d' ' -f9 | sed -e 's/<//g')
WIND_CHILL=$(echo $LINE | cut -d' ' -f10 | sed -e 's/<//g')
HEAT_IND=$(echo $LINE | cut -d' ' -f11 | sed -e 's/<//g')
ALT=$(echo $LINE | cut -d' ' -f12 | sed -e 's/<//g')
SEA_LEV=$(echo $LINE | cut -d' ' -f13 | sed -e 's/<//g')
PCPT_1HR=$(echo $LINE | cut -d' ' -f14 | sed -e 's/<//g')
PCPT_3HR=$(echo $LINE | cut -d' ' -f15 | sed -e 's/<//g')
PCPT_6HR=$(echo $LINE | cut -d' ' -f16 | sed -e 's/<//g')

OUT="$DATE, $NOW, $WIND, $VIS, $WN, $SKY_COND, $AIR_T, $DWPNT, $MAX6HR, $MIN6HR, $HUMID, $WIND_CHILL, $HEAT_IND, $ALT, $SEA_LEV, $PCPT_1HR, $PCPT_3HR, $PCPT_6HR"

sed -i "s/INSERT/$OUT/g" $1
~                             

In [ ]:
### StartDAQ.sh

#!/bin/bash
  
echo $1
DAY=$(date +%D)
TIME=$(date +%T)
echo "$DAY $TIME" > $1
echo "Day of month, Time of data collection, Wind speed (kph), Visibility (km), Weather, Sky COnditions, Air Temp (C), Dewpoint (C), 6 Hour Max Temp, 6 Hour Min Temp, Relative Humidity (%), Wind Chill (C), Heat Index (C), Alitimeter Pressure (cm), Sea Level (mb), Precipitation 1h r (cm), Precipitation 3hr (cm), Precipitation 6h (cm)" >> $1
echo "INSERT" >> $1
echo "---" >> $1
echo "Incident number, relative time since start, timestamp" >> $1
~                                                                                                                                       
~                                                                           

### [RN] 3:15 

I will check back in an hour to see if the first data acquisition is started. i will keep checking back and periodically update the git data until I can think of an easy eay to automate it. 